<a href="https://colab.research.google.com/github/isaacwanderers/artcreate/blob/main/%23GundGallery_Beta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# #GundGallery

Wellcome to our Beta version of our #GundGallery art developer and interpreter tool.

This program uses Machine Learning Neural Network to interpret an art piece to text, create an art piece from text, or create a new art piece from your chosen art piece with your highlighted element of the piece.
This program also includes a mutation algorithm that continuously merge and augment artworks and texts over multiple cycles (generations) to create thousands of artworks or text interpretations, all of which is filtered for the ones that best matches your target/chosen topic or artwork.

For faster results we encourage you to secure stable internet connections and a Google Colab Pro account. But a Google Colab Pro account is totally not necessary.

**Disclaimer:** Machine Learning Neural Network is a highly complicated mathematical simulator that is equivalent to a black box. If you have any suggestions or find interesting patterns in the function of this program, please let our development team know at:
pham2@kenyon.edu (Minh Pham '23), beshentseva2@kenyon.edu ( Irina Beshentseva '24), tjandra1@kenyon.edu (Isaac Tjandra '24), or niehoff1@kenyon.edu (Mart Niehoff '23)

## Instructions

1) Click the play button titled "System Set-up" and follow isntructions to start the system - About 5 minutes.

2) Select your setting options in the "Art Interpreter & Developer" form:
- genre: choose the genre of art you want to work in.

- save_each: ask the program to display to you intermediate results after n number of cycles/generations. This will slightly slow down your program if you set it too frequently

- generations: choose the total number of cycles/generations you want the program to run for. This will significantly slow down your program if you choose a number too high.


3) Confirm your settings and press the start button next to the title "Art Interpreter & Developer".

4) Follow the program instructions.

5) Download your final image if you would like. Images will be downloaded in .jpeg format and text will be downloaded in .txt format.

##Notes

1) If you forget to download your previous results, they are still stored in the "result" file in the "Files" tab of your left tool collumn.

2) We do not recommend unauthorized changes to the code. All chagnes that you made in this code will not be saved.


In [ ]:
#@title <-------System Set-up: Click the Button 
import os, sys

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout
        
        
with HiddenPrints():
    import subprocess
    import re

    nvcc = subprocess.check_output(["nvcc", "--version"]).decode("utf-8")
    version = re.findall("release (\d+\.\d+)", nvcc)[0]

    pytorch_suffix = {
        "10.0": "+cu100",
        "10.1": "+cu101",
        "10.2": "",
    }

    pytorch_version = "1.7.1" + (pytorch_suffix[version] if version in pytorch_suffix else "+cu110")
    torchvision_version = "0.8.2" + (pytorch_suffix[version] if version in pytorch_suffix else "+cu110")

    %cd /content
    !mkdir result
    !git clone https://github.com/isaacwanderers/artcreate.git
    %cd artcreate

    try:
      import torch
    except:
      !pip install torch=={pytorch_version} -f https://download.pytorch.org/whl/torch_stable.html

    try:
      import ipyplot
    except:
      !pip install ipyplot

    try:
      import torchvision
    except:
      !pip install torchvision=={torchvision_version} -f https://download.pytorch.org/whl/torch_stable.html

    try:
      import wget
    except:
      !pip install wget

    import urllib3
restart = 0
if urllib3.__version__ != "1.25.11":
    restart = 1
    print("BIGAN Neural Network requires urllib3 version 1.25.11.\nThe system will replace the current version of urllib3 with urllib3 1.25.11 and restart runtime.\nThis will no affect your future Colab work space, only this one program's environment.\nPlease re-initialize system set-up after restart!")
    
if restart ==1:
    with HiddenPrints():
        !pip install pytorch_pretrained_biggan==0.1.1 pymoo==0.4.2.1 kornia==0.4.1 ftfy==5.8 tensorboard==2.4.1
        import os
        os.kill(os.getpid(), 9)

import argparse
import os
import torch
import numpy as np
import pickle
import glob
import random
import array
import PIL.Image
import ipyplot
import matplotlib.pyplot as plt
import time
from os.path import exists
from pymoo.optimize import minimize
from pymoo.algorithms.so_genetic_algorithm import GA
from pymoo.factory import get_algorithm, get_decision_making, get_decomposition
from pymoo.visualization.scatter import Scatter
from IPython.display import Image, display
import torchvision
import urllib.request
import wget

from config import get_config
from problem import GenerationProblem
from operators import get_operators
import warnings
warnings.filterwarnings('ignore')

def display_genre_images(genre):
  image_files = sorted(glob.glob("/content/artcreate/gpt2_images/" + genre +"/*.jpg")) 
  all_images = []
  all_file_names = []

  for f in image_files:
    #enumerate() returns the index number/order number of the images and the root pathway of the image.
    try:
      image = PIL.Image.open(f)
      #open each image as an Image object class, which was imported above from the PIL module.
      all_images.append(image)
      all_file_names.append(f)
    except:
      print("error reading", f)

  file_name = [i.rpartition('/')[2] for i in all_file_names]
  ipyplot.plot_images(all_images, file_name, img_width=150)
  if len(file_name) <= 20:
    time.sleep(15)
  else:
    time.sleep(30)
  return file_name

import IPython
import uuid
from google.colab import output
from google.colab import files

class InvokeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback

  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)

    template = """<button id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html

def download_file():
  files.download(result_path)
  print("File Successfully Downloaded!")

def save_callback(algorithm):
  global iteration
  global config

  iteration += 1
  if iteration % config.save_each == 0 or iteration == config.generations:
      if config.problem_args["n_obj"] == 1:
          sortedpop = sorted(algorithm.pop, key=lambda p: p.F)
          X = np.stack([p.X for p in sortedpop])  
      else:
          X = algorithm.pop.get("X")
      
      ls = config.latent(config)
      ls.set_from_population(X)

      with torch.no_grad():
          generated = algorithm.problem.generator.generate(ls, minibatch=config.batch_size)
          name = "genetic-it-%d.jpg" % (iteration) if iteration < config.generations else "genetic-it-final.jpg"

          if config.task == "txt2img":
              algorithm.problem.generator.save(generated, os.path.join(config.tmp_folder, name))
              display(Image(os.path.join(config.tmp_folder, name)))
          elif config.task == "img2txt":
              print("\n".join(generated))

print("The system has finished setting up!\nYou are good to start using the development modes.\nCaution: Remember to rerun the set up by clicking the button above whenever this line disappears.")

The system has finished setting up!
You are good to start using the development modes.
Caution: Remember to rerun the set up by clicking the button above whenever this line disappears.


In [ ]:
#@title <-------Art Interpreter & Developer
#@markdown Choose your settings below and start the program.

#@markdown The program may ask for your input below as it executes.

#@markdown Each development mode takes 15 - 30 minutes to complete. The program will slow down if you run it more than 10 times at once.

#@markdown If the program slows down, closing out of the browser and reopen the program may help.
development_mode = "interpret Gund Gallery art to text" #@param ["interpret Gund Gallery art to text", "create art from text", "create new art from Gund Gallery art"]
genre = "Mixed Media Art" #@param ["Drawing", "Mixed Media Art", "Painting", "Photograph", "Print", "Sculpture"]
save_each =  "200"#@param ["100", "200", "300", "400", "500", "600", "700", "800", "900", "1000"]
generations =  "400"#@param ["100", "200", "300", "400", "500", "600", "700", "800", "900", "1000"]
save_each = int(save_each)
generations = int(generations)
if development_mode == "interpret Gund Gallery art to text":
  file_exists = exists('/content/artcreate/gpt2/weights/gpt2-pytorch_model.bin')
  if not file_exists:
    print('Setting up image recognition system for the first time')
    ! chmod 755 ./download-weights.sh
    ! ./download-weights.sh GPT2
    print('Finish setting up image recognitiion system!')
  print("---All images available in genre---")
  file_name = display_genre_images(genre)
  target_image = input("Input the file name of the image you want to interpret: ")
  while target_image.rpartition('.')[2] != "jpg":
    print("File input error: Please include the file extension!")
    target_image = input("Re-input the file name of the image you want to interpret: ")
  while target_image not in file_name:
    print("File input error: Please type your file name correctly!")
    target_image = input("Re-input the file name of the image you want to interpret: ")
  target = "/content/artcreate/gpt2_images/" + genre + "/" + target_image
  print("---Chosen Image---")
  display(Image(target))
  print(target_image)
  config = "GPT2" 
  generations = generations*3
  print("---Interpreting your image---")
elif development_mode == "create art from text":
  target = input("Input your target phrase: ") 
  config = "DeepMindBigGAN512"
  target = genre.lower() + " with " + target
  print("---Generating your image---")
else:
  file_exists = exists('/content/artcreate/gpt2/weights/gpt2-pytorch_model.bin')
  if not file_exists:
    print('Setting up image recognition system for the first time')
    ! chmod 755 ./download-weights.sh
    ! ./download-weights.sh GPT2
    print('Finish setting up image recognitiion system!')
  file_name = display_genre_images(genre)
  target_image = input("Input the file name of the image you want to develop: ")
  while target_image.rpartition('.')[2] != "jpg":
    print("File input error: Please include the file extension!")
    target_image = input("Re-input the file name of the image you want to develop: ")
  while target_image not in file_name:
    print("File input error: Please type your file name correctly!")
    target_image = input("Re-input the file name of the image you want to develop: ")
  target = "/content/artcreate/gpt2_images/" + genre + "/" + target_image
  print("---Chosen Image---")
  display(Image(target))
  print(target_image)
  element = input("What element stands out the most to you? (1-2 words) ")  
  config = "GPT2"
   
  print("---Interpreting your image---")
  config = argparse.Namespace(
      config=config,
      target=target,
      device="cuda",
      generations=generations*3,
      save_each=save_each,
      tmp_folder="./tmp"
  )

  vars(config).update(get_config(config.config))


  iteration = 0          

  problem = GenerationProblem(config)
  operators = get_operators(config)

  if not os.path.exists(config.tmp_folder): os.mkdir(config.tmp_folder)

  algorithm = get_algorithm(
      config.algorithm,
      pop_size=config.pop_size,
      sampling=operators["sampling"],
      crossover=operators["crossover"],
      mutation=operators["mutation"],
      eliminate_duplicates=True,
      callback=save_callback,
      **(config.algorithm_args[config.algorithm] if "algorithm_args" in config and config.algorithm in config.algorithm_args else dict())
  )

  res = minimize(
      problem,
      algorithm,
      ("n_gen", config.generations),
      save_history=False,
      verbose=True,
  )


  pickle.dump(dict(
      X = res.X,
      F = res.F,
      G = res.G,
      CV = res.CV,
  ), open(os.path.join(config.tmp_folder, "genetic_result"), "wb"))


  if config.problem_args["n_obj"] == 1:
      X = np.atleast_2d(res.X)
  else:
      try:
          result = get_decision_making("pseudo-weights", [0, 1]).do(res.F)
      except:
          print("Warning: cant use pseudo-weights")
          result = get_decomposition("asf").do(res.F, [0, 1]).argmin()
      X = res.X[result]
      X = np.atleast_2d(X)


  ls = config.latent(config)
  ls.set_from_population(X)

  with torch.no_grad():
      generated = problem.generator.generate(ls)

  problem.generator.save(generated, ("/content/result/output.txt"))
  print("---Finished interpreting your image---")
  text_file = open("/content/result/output.txt", "r")
  interpret = text_file.read()
  text_file.close()
  config = "DeepMindBigGAN512"
  target = genre.lower() + " of " + interpret.rpartition("of")[2] + element*3
  !rm /content/result/output.txt
  #End of the first text to image intepretation phase of the 3rd development mode.
  print("---Generating your image---")

config = argparse.Namespace(
    config=config,
    target=target,
    device="cuda",
    generations=generations,
    save_each=save_each,
    tmp_folder="./tmp"
)

vars(config).update(get_config(config.config))

iteration = 0     

problem = GenerationProblem(config)
operators = get_operators(config)

if not os.path.exists(config.tmp_folder): os.mkdir(config.tmp_folder)

algorithm = get_algorithm(
    config.algorithm,
    pop_size=config.pop_size,
    sampling=operators["sampling"],
    crossover=operators["crossover"],
    mutation=operators["mutation"],
    eliminate_duplicates=True,
    callback=save_callback,
    **(config.algorithm_args[config.algorithm] if "algorithm_args" in config and config.algorithm in config.algorithm_args else dict())
)

res = minimize(
    problem,
    algorithm,
    ("n_gen", int(config.generations)),
    save_history=False,
    verbose=True,
)


pickle.dump(dict(
    X = res.X,
    F = res.F,
    G = res.G,
    CV = res.CV,
), open(os.path.join(config.tmp_folder, "genetic_result"), "wb"))


if config.problem_args["n_obj"] == 1:
    X = np.atleast_2d(res.X)
else:
    try:
        result = get_decision_making("pseudo-weights", [0, 1]).do(res.F)
    except:
        print("Warning: cant use pseudo-weights")
        result = get_decomposition("asf").do(res.F, [0, 1]).argmin()
    X = res.X[result]
    X = np.atleast_2d(X)

ls = config.latent(config)
ls.set_from_population(X)

with torch.no_grad():
    generated = problem.generator.generate(ls)

#Check to see if there are previous output files exist in the result directory.
#Previous outputs means that this is not the first instance of the program in this script.

output_names = sorted(glob.glob("/content/result/*"))
if len(output_names) ==0:
    n = 1
else:
    n = int(output_names[-1].rpartition('.')[0].rpartition('_')[2]) + 1

if config.task == "txt2img":
    print("---Finished generating your image!---")

    print("="*50)
    print("RESULT")
    print("="*50)

    result_path = "/content/result/output_" + str(n)+ ".jpg"
    problem.generator.save(generated, result_path)
    display(Image(result_path))

elif config.task == "img2txt":
    print("---Finished interpreting your image!---")

    print("="*50)
    print("RESULT")
    print("="*50)

    result_path = "/content/result/output_" + str(n)+ ".txt"
    print(generated)
    problem.generator.save(generated, result_path)

#This line is here because it cannot be put inside a loop
InvokeButton('Download Final Result', download_file) 

   53 |    5300 |  0.00000E+00 |  0.00000E+00 |       -0.283 |      -0.2455
   54 |    5400 |  0.00000E+00 |  0.00000E+00 |       -0.283 |      -0.2461
   55 |    5500 |  0.00000E+00 |  0.00000E+00 |       -0.283 |      -0.2465
   56 |    5600 |  0.00000E+00 |  0.00000E+00 |       -0.283 |      -0.2468
   57 |    5700 |  0.00000E+00 |  0.00000E+00 |       -0.283 |      -0.2468
   58 |    5800 |  0.00000E+00 |  0.00000E+00 |       -0.283 |      -0.2478
   59 |    5900 |  0.00000E+00 |  0.00000E+00 |       -0.283 |      -0.2482
   60 |    6000 |  0.00000E+00 |  0.00000E+00 |       -0.283 |      -0.2489
   61 |    6100 |  0.00000E+00 |  0.00000E+00 |      -0.2993 |      -0.2502
   62 |    6200 |  0.00000E+00 |  0.00000E+00 |      -0.2993 |      -0.2524
   63 |    6300 |  0.00000E+00 |  0.00000E+00 |      -0.2993 |      -0.2532
   64 |    6400 |  0.00000E+00 |  0.00000E+00 |      -0.2993 |      -0.2542
   65 |    6500 |  0.00000E+00 |  0.00000E+00 |      -0.2993 |      -0.2542
   66 |    6

KeyboardInterrupt: ignored